<a href="https://colab.research.google.com/github/rodzhttp/Pyspark1/blob/main/pyspark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**< Pyspark Project >**

In [ ]:
# Instala as bibliotecas necessárias
!pip install pyspark findspark

# Configura o ambiente para encontrar o Spark
import findspark
from google.colab import drive
findspark.init()

# Inicia a "Sessão Spark", que é o nosso motor para processar os dados
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("AnaliseVoos").getOrCreate()

print("✅ Sessão Spark iniciada com sucesso!")

✅ Sessão Spark iniciada com sucesso!


In [36]:
# O caminho para a pasta que contém seus arquivos
caminho_base = '/content/drive/My Drive/Colab Notebooks/Pyspark project/'

# Caminho completo para cada arquivo
caminho_flights = caminho_base + 'flights.csv'
caminho_airlines = caminho_base + 'airlines.csv'
caminho_airports = caminho_base + 'airports.csv'

# Lendo os arquivos para DataFrames
df_flights = spark.read.csv(caminho_flights, header=True, inferSchema=True)
df_airlines = spark.read.csv(caminho_airlines, header=True, inferSchema=True)
df_airports = spark.read.csv(caminho_airports, header=True, inferSchema=True)

display(df_flights.show(5))

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

None

In [54]:
from pyspark.sql.functions import avg, desc, round, col, when, concat, lit

# 1. Calcula a média de atraso
media_atrasos = df_flights.groupBy("AIRLINE") \
    .agg(
        round(avg("ARRIVAL_DELAY"), 2).alias("media_atraso_chegada")
    )

# 2. Junta com a tabela de companhias
resultado_atrasos = media_atrasos.join(
        df_airlines,
        media_atrasos.AIRLINE == df_airlines.IATA_CODE
    )

# 3. Formata a coluna de resultado condicionalmente
resultado_formatado = resultado_atrasos.withColumn("Resultado",
    when(col("media_atraso_chegada") >= 0,
         concat(col("media_atraso_chegada").cast("string"), lit(" minutos"))
    ).otherwise(
         col("media_atraso_chegada").cast("string")
    )
)

# 4. Ordena pelo valor numérico ANTES de selecionar as colunas finais
resultado_ordenado = resultado_formatado.orderBy(desc("media_atraso_chegada"))

# 5. Seleciona as colunas para exibição e mostra o resultado
print("Média de Atrasos na Chegada por Companhia Aérea:")
resultado_ordenado.select(
    df_airlines.AIRLINE.alias("NOME_DA_COMPANHIA"),
    "Resultado"
).show()
print("obs: Valores negativos significam que, na média, os voos chegaram adiantados")

Média de Atrasos na Chegada por Companhia Aérea:
+--------------------+-------------+
|   NOME_DA_COMPANHIA|    Resultado|
+--------------------+-------------+
|    Spirit Air Lines|14.47 minutos|
|Frontier Airlines...| 12.5 minutos|
|     JetBlue Airways| 6.68 minutos|
|Atlantic Southeas...| 6.59 minutos|
|American Eagle Ai...| 6.46 minutos|
|Skywest Airlines ...| 5.85 minutos|
|United Air Lines ...| 5.43 minutos|
|      Virgin America| 4.74 minutos|
|Southwest Airline...| 4.37 minutos|
|     US Airways Inc.| 3.71 minutos|
|American Airlines...| 3.45 minutos|
|Hawaiian Airlines...| 2.02 minutos|
|Delta Air Lines Inc.| 0.19 minutos|
|Alaska Airlines Inc.|        -0.98|
+--------------------+-------------+

obs: Valores negativos significam que, na média, os voos chegaram adiantados


In [53]:
from pyspark.sql.functions import avg, round, concat, lit, lpad, col, when

print("Média de Atraso na Chegada por Mês (formatado):")

# 1. Cria a coluna 'ANO_MES'
df_com_data = df_flights.withColumn("ANO_MES",
    concat(col("YEAR"), lit("-"), lpad(col("MONTH"), 2, "0"))
)

# 2. Calcula a média de atraso
atraso_calculado = df_com_data.groupBy("ANO_MES") \
    .agg(round(avg("ARRIVAL_DELAY"), 2).alias("media_de_atraso"))

# 3. Formata a coluna de resultado condicionalmente
resultado_final = atraso_calculado.withColumn("Resultado",
    when(col("media_de_atraso") >= 0,
         concat(col("media_de_atraso").cast("string"), lit(" minutos"))
    ).otherwise(
         col("media_de_atraso").cast("string") # Para valores negativos, mostra só o número
    )
).orderBy("ANO_MES")

resultado_final.select("ANO_MES", "Resultado").show()
print("obs: Valores negativos significam que, na média, os voos chegaram adiantados")

Média de Atraso na Chegada por Mês (formatado):
+-------+------------+
|ANO_MES|   Resultado|
+-------+------------+
|2015-01|5.81 minutos|
|2015-02|8.32 minutos|
|2015-03|4.92 minutos|
|2015-04|3.16 minutos|
|2015-05|4.49 minutos|
|2015-06| 9.6 minutos|
|2015-07|6.43 minutos|
|2015-08|4.61 minutos|
|2015-09|       -0.77|
|2015-10|       -0.78|
|2015-11| 1.1 minutos|
|2015-12|6.09 minutos|
+-------+------------+

obs: Valores negativos significam que, na média, os voos chegaram adiantados


In [55]:
from pyspark.sql.functions import avg, round, col, when, concat, lit, asc

# 1. Calcula a média de atraso por dia da semana
atraso_por_dia = df_flights.groupBy("DAY_OF_WEEK") \
    .agg(round(avg("ARRIVAL_DELAY"), 2).alias("media_atraso"))

# 2. Mapeia o número do dia para o nome correspondente
mapeamento_dia = atraso_por_dia.withColumn("Dia da Semana",
    when(col("DAY_OF_WEEK") == 1, "Domingo")
    .when(col("DAY_OF_WEEK") == 2, "Segunda-feira")
    .when(col("DAY_OF_WEEK") == 3, "Terça-feira")
    .when(col("DAY_OF_WEEK") == 4, "Quarta-feira")
    .when(col("DAY_OF_WEEK") == 5, "Quinta-feira")
    .when(col("DAY_OF_WEEK") == 6, "Sexta-feira")
    .otherwise("Sábado")
)

# 3. Formata a coluna de resultado condicionalmente
resultado_formatado = mapeamento_dia.withColumn("Resultado",
    when(col("media_atraso") >= 0,
         concat(col("media_atraso").cast("string"), lit(" minutos"))
    ).otherwise(
         col("media_atraso").cast("string")
    )
)

# 4. Ordena pelo menor atraso (melhor dia primeiro) e exibe
print("Média de Atraso por Dia da Semana (ordenado do melhor para o pior):")
resultado_formatado.orderBy(asc("media_atraso")).select("Dia da Semana", "Resultado").show()

Média de Atraso por Dia da Semana (ordenado do melhor para o pior):
+-------------+------------+
|Dia da Semana|   Resultado|
+-------------+------------+
|  Sexta-feira|1.85 minutos|
|  Terça-feira|3.85 minutos|
|       Sábado|3.96 minutos|
|Segunda-feira|4.25 minutos|
| Quinta-feira|4.76 minutos|
| Quarta-feira|5.67 minutos|
|      Domingo| 6.0 minutos|
+-------------+------------+

